In [ ]:
import pandas as pd
import numpy as np
import math
import time
import os

In [ ]:
LARGE_DATASET = True

In [ ]:
if LARGE_DATASET:
    movie_path = "ml-latest/movies.csv"
    tags_path = "ml-latest/tags.csv"
    ratings_path = "ml-latest/split/ratings_testing.csv"
    movie_profiles_path = "/media/karol/E8B625B3B6258364/calculated/movie_profiles"
    user_profiles_path = "/media/karol/E8B625B3B6258364/calculated/user_profiles/"
    movie_matrix_path = "/media/karol/E8B625B3B6258364/calculated/movie_matrix"
else:
    movie_path = "ml-latest-small/movies.csv"
    tags_path = "ml-latest-small/tags.csv"
    ratings_path = "ml-latest-small/split/ratings_testing.csv"
    movie_profiles_path = "/media/karol/E8B625B3B6258364/calculated-small/movie_profiles"
    user_profiles_path = "/media/karol/E8B625B3B6258364/calculated-small/user_profiles/"

In [ ]:
def read_user_profile(userId):
    directory = user_profiles_path + str(userId // 1000) + "/" + str(userId)
    user_profile = pd.read_csv(directory,  header=None, usecols=[1,2,3])
    return user_profile.rename(columns={1:"tag",2:"u_weight",3:"userId"})

def get_movie_profile(movieId):
    return movie_profiles[movie_profiles['movieId'] == movieId]

# Predict user's rating on test data

Profile filmów oraz użytkowników zapisane są w postaci list tagów oraz ich wag związanych z danymi filmami i użytkownikami, sposób ten jest oszczędny jeśli chodzi o zajęte miejsce na dysku, jednak obliczenia wektorowe na takiej strukturze są zdecydowanie za wolne. Z tego powodu wszystkie profile filmów zostają najpierw zamienione na postać macierzy, a wszystkie profile użytkownika po kolei zostają zamienione w postać wektorową.

In [ ]:
# get_movie_profile_matrix: dla podanych indeksów filmów tworzona jest macierz,
# w której każda kolumna reprezentuje wektor film, a każdy wiersz odpowiada danemu tagowi 
def get_movie_profile_matrix(movie_indices):
    if len(movie_indices) == 1:
        mid = movie_indices.iloc[0]
        return get_movie_profile(mid).drop(['movieId'],1).rename(columns={"m_weight": mid})
    
    matrix_left = get_movie_profile_matrix(movie_indices[:len(movie_indices)//2])
    matrix_right = get_movie_profile_matrix(movie_indices[len(movie_indices)//2:])
    matrix = pd.merge(matrix_left,matrix_right , on='tag', how='outer', sort=False)
    return matrix

# get_similarity: funkcja obliczająca odległość kosinusową między wektorem użytkownika
# a macierzą filmów, z założeniem że wszystkie wektory mają długość 1
def get_similarity(userId, movies_df):
    i = read_user_profile(userId)
    i['u_weight'] = i['u_weight'] / np.sqrt(np.square(i['u_weight']).sum())
    similarity = pd.merge(i, movies_df, on='tag', how='inner', sort=False)
    return similarity.iloc[:,3:].mul(i['u_weight'],0).sum(0)

In [ ]:
movie_profiles = pd.read_csv(movie_profiles_path).rename(columns={'weight':'m_weight'})
ratings_data = pd.read_csv(ratings_path)

In [ ]:
movies = ratings_data['movieId'].drop_duplicates()

In [ ]:
LOAD_MATRIX = False

In [ ]:
if LOAD_MATRIX:
    movie_matrix = pd.read_csv('/media/karol/E8B625B3B6258364/calculated/movie_matrix')
else:
    movie_matrix = get_movie_profile_matrix(movies)

Obliczenie rekomendacji dla użytkowników.
Dla każdego użytkownika tworzona jest lista indeksów filmów, posortowane wg. kryterium
odległości, każda lista zapisywana jest w osobnym pliku dla wygody wczytywania/edycji
konkretnego użytkownika

In [ ]:
start = time.time()
x = 0
for userId in ratings_data['userId'].drop_duplicates():
        print(userId)
        f = open("calculated/content_based_recommendations/" + str(userId), "w")
        recomendations = pd.DataFrame()
        recomendations['movie'] = get_similarity(userId,movie_matrix).sort_values(ascending=False).index
        recomendations.to_csv(f, header=None ,index=False)
print(time.time() - start)

# Mały zestaw dany został przetworzony w mniej niż 4sekundy

w dużym zestawie każdy użytkownik wymaga około sekundy na obliczenie, ostatecznie obliczone
zostały rekomendacje dla około 10tyś. losowych użytkowników